In [80]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from collections import Counter
from tqdm import tqdm
import pandas as pd

In [81]:
df = pd.read_csv('training_freek_feateng.csv')

In [82]:
columns = ['srch_id', 'booking_bool', 'click_bool', 'position', 'random_bool', 'promotion_flag', 'srch_length_of_stay', 'prop_id']
df = df[columns].copy()

In [83]:
df.head()

,srch_id,booking_bool,click_bool,position,random_bool,promotion_flag,srch_length_of_stay,prop_id
0,1,0,0,27,1,0,1,893
1,1,0,0,26,1,0,1,10404
2,1,0,0,21,1,0,1,21315
3,1,0,0,34,1,0,1,27348
4,1,0,0,4,1,0,1,29604


In [92]:
X = df.drop(['booking_bool','position','random_bool','prop_id'], axis=1)
y = df['booking_bool']

In [93]:
df.columns

Index(['srch_id', 'booking_bool', 'click_bool', 'position', 'random_bool',
       'promotion_flag', 'srch_length_of_stay', 'prop_id'],
      dtype='object')

In [94]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01)

In [95]:
# save click_bool and srch_id from test
click_list = X_test['click_bool']
srchid_list = X_test['srch_id']


# remove click_bool and srch_id from train and test
X_train = X_train.drop(['click_bool', 'srch_id'], axis=1)
X_test = X_test.drop(['click_bool', 'srch_id'], axis=1)

In [96]:
X_train.columns

Index(['promotion_flag', 'srch_length_of_stay'], dtype='object')

In [97]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [98]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
y_pred = []
for i in tqdm(range(len(X_test))):
    pred = knn.predict_proba([X_test[i]])
    y_pred.append(pred[0])

  5%|▌         | 2668/49584 [00:11<03:20, 234.46it/s]

In [91]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.97224


In [92]:
# Count the occurrences of each element
counts = Counter(y_pred)

# Print the counts
print("Count of 0's:", counts[0])
print("Count of 1's:", counts[1])

Count of 0's: 24954
Count of 1's: 46


# Predicting on test

In [11]:
test_df =  pd.read_csv('test_freek_feateng.csv')

In [12]:
prop_ids = test_df['prop_id']
srch_ids = test_df['srch_id']
test_df = test_df.drop(['prop_id', 'srch_id'],axis=1)

In [13]:
test_df.columns

Index(['site_id', 'visitor_location_country_id', 'prop_country_id',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'orig_destination_distance', 'random_bool',
       'month', 'prop_starrating_location_interaction',
       'price_historical_interaction', 'total_time_duration',
       'highly_rated_prop', 'season', 'total_people'],
      dtype='object')

In [14]:
y_pred = []

# Iterate over the rows of the test DataFrame with tqdm for progress bar
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    X_test = row  # Use the entire row for prediction
    pred = knn.predict([X_test])  # Assuming you have trained the KNN model and stored it as `knn`
    y_pred.append(pred[0])

  0%|          | 0/4959183 [00:00<?, ?it/s]


ValueError: X has 27 features, but KNeighborsClassifier is expecting 26 features as input.

In [100]:
prediction_df = pd.concat( [srch_ids, prop_ids] , axis=1)

In [101]:
prediction_df

,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
49995,3261,73456
49996,3261,84390
49997,3261,87468
49998,3261,93466


In [106]:
prediction_df['prob'] = y_pred[:,1]

TypeError: list indices must be integers or slices, not tuple